In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import math

In [3]:
telecom_df = pd.read_csv('telecom_churn_data.csv')

In [4]:
telecom_df.info()
#Most are numeric, some of object type. Check these


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99999 entries, 0 to 99998
Columns: 226 entries, mobile_number to sep_vbc_3g
dtypes: float64(179), int64(35), object(12)
memory usage: 172.4+ MB


In [5]:
#print(telecom_df.select_dtypes(include=['object']).head())
#These are all of type data, of format mm/dd/yyyy, conver to dates
#for all date columns, keep only the day of the month, since month is present in the
#column name, year is common for all.
date_columns = list(telecom_df.select_dtypes(include=['object']).columns)
for column in date_columns:
    telecom_df[column] = pd.to_datetime(telecom_df[column], format='%m/%d/%Y')
    telecom_df[column] = telecom_df[column].dt.day

In [76]:
#Those who have recharged with an amount more than or equal to X, 
#where X is the 70th percentile of the average recharge amount in the first two months (the good phase).
'''
print(telecom_df['total_rech_amt_6'].isnull().sum()) 0;
print(telecom_df['total_rech_amt_7'].isnull().sum()) 0;
print(telecom_df['total_rech_num_6'].isnull().sum()) 0;
print(telecom_df['total_rech_num_7'].isnull().sum()) 0;
'''
#since, all values for the columns used below are non-nulls, we need not do any imputation here
if 'av_rech_67' not in list(telecom_df.columns):
    telecom_df['av_rech_67'] =  (telecom_df['total_rech_amt_6']+telecom_df['total_rech_amt_7'])/(telecom_df['total_rech_num_6']+telecom_df['total_rech_num_7'])
    avg_70_quantile = telecom_df.av_rech_67.quantile(q=0.7)
    telecom_df = telecom_df[telecom_df.av_rech_67>= avg_70_quantile]
print ('Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: '+ str(len(telecom_df)))


Data is now at 70th quantile of average recharge for the first 2 months. Total rows remaining: 29944


In [77]:
telecom_df.isnull().sum().describe()
# telecom_df.isnull().sum()>0).sum()
# 166 columns with missing values
# A lot of values are missing in many colums. Cannot remove such rows. Will have to find a way to impute

count      227.000000
mean      4205.832599
std       7207.678037
min          0.000000
25%          0.000000
50%        989.000000
75%       2083.000000
max      19826.000000
dtype: float64

In [78]:
#Find columns with only single non-null value
redundant_cols = [col for col in telecom_df.columns if telecom_df[col].nunique()==1]
telecom_df = telecom_df.drop(redundant_cols, axis=1)
print('Following columns were removed: \n',redundant_cols)


Following columns were removed: 
 ['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'last_date_of_month_9', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_og_t2c_mou_9', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8', 'std_ic_t2o_mou_9']


In [99]:
#Find rows with null dates
for month in [6,7,8,9]:
    last_rech_col = 'date_of_last_rech_'+str(month)
    num_recharges_col = 'total_rech_num_'+str(month)
    nullrechdate_df = telecom_df[telecom_df['date_of_last_rech_9'].isnull()]
    if (len(nullrechdate_df[~nullrechdate_df.total_rech_num_9==0])>0):
        print('Nulls found for date in month:', month)
    else:
        print('All nulls in month ',month, ' correspond to no recharges done.')
    
#Hence, nans in date_of_last_rech_ columns are all valid. Giving value > 32 so that we know it is invalid


#dates can be changed to categorical values

#NaN can be assumed to be Recharge not done 

# def process(x):
#     if type(x) is float:
#         return 0
#     else: 
#         return 1

# for col in dc:
#     dat[col] = dat[col].apply(process)

All nulls in month  6  correspond to no recharges done.
All nulls in month  7  correspond to no recharges done.
All nulls in month  8  correspond to no recharges done.
All nulls in month  9  correspond to no recharges done.


In [1]:
telecom_df.isnull().sum()

NameError: name 'telecom_df' is not defined

In [106]:
telecom_df.isnull().sum()>0).sum()


['roam_ic_mou_6',
 'roam_ic_mou_7',
 'roam_ic_mou_8',
 'roam_ic_mou_9',
 'roam_og_mou_6',
 'roam_og_mou_7',
 'roam_og_mou_8',
 'roam_og_mou_9']

In [87]:
dat[dc].head()

,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,date_of_last_rech_data_6,date_of_last_rech_data_7,date_of_last_rech_data_8,date_of_last_rech_data_9
0,1,1,1,1,1,1,1,0
1,1,1,1,1,0,1,1,0
2,1,1,1,1,0,0,0,1
3,1,1,1,1,0,0,0,0
4,1,1,1,1,1,0,0,0


In [43]:
telecom_df['std_ic_t2o_mou_9'].unique()

array([nan,  0.])